In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install datasets
!pip install transformers
!pip install peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


In [ ]:
# !pip install --upgrade --no-cache-dir git+https://github.com/Shannu3766/bi_influence.git

In [ ]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")
# logging.set_verbosity_error()

In [ ]:
# from datasets import load_dataset, DatasetDict, Dataset

# from transformers import (
#     AutoTokenizer,
#     AutoConfig,
#     AutoModelForSequenceClassification,
#     DataCollatorWithPadding,
#     TrainingArguments,
#     Trainer)

# from peft import PeftModel, PeftConfig, get_peft_model, LoraConfig
# import evaluate
# import torch
# import numpy as np
# import os
# from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
# from peft import get_peft_model, LoraConfig, TaskType
# from torch.utils.data import DataLoader
# # from adaptive_lora.callbacks import AdaptiveLoRACallback
# import numpy as np
# print("torch:", torch.__version__)
# print("cuda devices:", torch.cuda.device_count())

In [1]:
import os
import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer
)
from peft import get_peft_model, LoraConfig, TaskType
os.environ["WANDB_DISABLED"] = "true"


In [2]:
MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
TASK = "rte"
LR = 2e-4
NUM_EPOCHS = 1
BATCH_SIZE = 8 
MAX_LEN = 256  

In [3]:
dataset = load_dataset("glue", TASK)
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


In [4]:
from transformers import DataCollatorWithPadding

# Make sure pad token exists for TinyLlama
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

MAX_LEN = 256  # enough for RTE, avoids over-long sequences


def clean_text(s):
    if s is None:
        return ""
    # strip + collapse multiple spaces/newlines
    return " ".join(str(s).strip().split())


def preprocess_function(examples):
    premises = [clean_text(p) for p in examples["sentence1"]]
    hypotheses = [clean_text(h) for h in examples["sentence2"]]

    # Clear, instruction-like prompt with an explicit "Answer:"
    inputs = [
        (
            f"Premise: {p}\n"
            f"Hypothesis: {h}\n"
            f"Question: Does the premise entail the hypothesis?\n"
            f"Answer:"
        )
        for p, h in zip(premises, hypotheses)
    ]

    tokenized = tokenizer(
        inputs,
        truncation=True,
        max_length=MAX_LEN,
        padding=False,        # ❗ no fixed padding here
    )
    return tokenized


# Map over the dataset
encoded_dataset = dataset.map(preprocess_function, batched=True)

# Drop original text columns
encoded_dataset = encoded_dataset.remove_columns(["sentence1", "sentence2", "idx"])

# Make sure labels column is named "labels"
if "label" in encoded_dataset["train"].column_names and "labels" not in encoded_dataset["train"].column_names:
    encoded_dataset = encoded_dataset.rename_column("label", "labels")

encoded_dataset.set_format("torch")

# Use dynamic padding at batch level
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    padding="longest",
)


Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2,
    device_map="auto",
    torch_dtype=torch.bfloat16, 
    trust_remote_code=True
)

`torch_dtype` is deprecated! Use `dtype` instead!
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False 

In [7]:
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=8,                    
    lora_alpha=32,          
    lora_dropout=0.05,
    bias="none",
    target_modules=[   
        "q_proj", "k_proj", "v_proj", "o_proj", 
        "gate_proj", "up_proj", "down_proj"
    ]
)

In [8]:
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(
    output_dir=f"{MODEL_ID}-RTE-v2",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LR,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    logging_steps=20,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=False,             
    bf16=True,      
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

print("🚀 Starting Training...")
trainer.train()

trainable params: 6,311,936 || all params: 1,040,828,416 || trainable%: 0.6064


/tmp/ipython-input-1242714479.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.


🚀 Starting Training...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.541700,0.447307,0.797834


TrainOutput(global_step=312, training_loss=0.589604472502684, metrics={'train_runtime': 959.4214, 'train_samples_per_second': 2.595, 'train_steps_per_second': 0.325, 'total_flos': 2714277805424640.0, 'train_loss': 0.589604472502684, 'epoch': 1.0})

In [ ]:
print("📊 Final Evaluation:")
results = trainer.evaluate()
print(results)